In [1]:
import sys
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, roc_auc_score, roc_curve
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Apologies for the inconcise code. I was attempting to add on to a partners previous code that didn't use cross validation at the last minute so I didn't have time to make concise functions.

In [2]:
# Read the training dataset from 'train_df.csv' into a DataFrame
train_df = pd.read_csv('train_df')

In [3]:
#Selected Features (Top 10):
important_column_10= [
    'Age Group', 'Num of Bad Mental Health Days', 'Hours of Sleeping', 
    'Arthritis', 'Days Drinking', 'Pneumonia Vaccine', 'Tetanus Last 10 Years', 
    'Income', 'BMI', 'Ethnicity_White', 'Cancer'
    ]
# Select the important columns for RFE from the training DataFrame
train_df_10 = train_df[important_column_10]

# Separate the features and target variable from the training dataset
X_train_10= train_df_10.drop('Cancer', axis=1)
y_train_10 = train_df_10['Cancer']

In [4]:
#Selected Features (Top 20):
important_column_20 = [
    'Gender', 'Age Group', 'Num of Bad Mental Health Days', 'Years Since Last Checkup', 
    'Hours of Sleeping', 'Arthritis', 'Married', 'Deaf', 'Age Started Smoking', 
    'Cigarettes per Day', 'Days Drinking', 'Flu Vaccine', 'Pneumonia Vaccine', 
    'Tetanus Last 10 Years', 'Had COVID', 'Metropolitan', 'Income', 'Insulin', 
    'BMI', 'Ethnicity_White', 'Cancer'
]
# Select the important columns for RFE from the training Dataframe
train_df_20 = train_df[important_column_20]

# Separate the features and target variable from the training dataset
X_train_20= train_df_20.drop('Cancer', axis=1)
y_train_20 = train_df_20['Cancer']

In [5]:
#Selected Features (Top 30):
important_column_30 = [
    'Gender', 'Age Group', 'Num of Bad Mental Health Days',
    'Years Since Last Checkup', 'Exercise in Past 30 Days',
    'Hours of Sleeping', 'Heart Disease', 'Asthma', 'Depression',
    'Kidney Disease', 'Arthritis', 'Diabetes', 'Married', 'Deaf', 'Blind',
    'Age Started Smoking', 'Cigarettes per Day', 'Days Drinking',
    'Flu Vaccine', 'Pneumonia Vaccine', 'Tetanus Last 10 Years',
    'Had COVID', 'Metropolitan', 'Income', 'Insulin', 'BMI',
    'Ethnicity_Hispanic', 'Ethnicity_White', 'Education_attended_college',
    'Education_graduated_college', 'Cancer'
]

# Select the important columns for RFE from the training Dataframe
train_df_30 = train_df[important_column_30]

# Separate the features and target variable from the training dataset
X_train_30= train_df_30.drop('Cancer', axis=1)
y_train_30 = train_df_30['Cancer']

In [6]:
# Separate the features and target variable from the training dataset
X_train_41= train_df.drop('Cancer', axis=1)
y_train_41 = train_df['Cancer']

## Top 41 Features

In [7]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# NB pipeline
nb_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('nb', GaussianNB())
])

# NB Parameter Grid
# Using optimal parameters from previous gridsearch
param_grid = {
    'nb__var_smoothing': [3.125], #using optimal parameter
}

# Grid search
grid_search = GridSearchCV(estimator=nb_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_41, y_train_41)

#print("Results Table: ", pd.DataFrame(grid_search.cv_results_))
#pd.set_option('display.max_columns',None)
print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time

Best parameters found:  {'nb__var_smoothing': 3.125}
Cross-validation recall:  0.8386
Cross-validation recall std:  0.0067
Cross-validation precision:  0.1821
Cross-validation precision std:  0.001
Cross-validation f1 score:  0.2992
Cross-validation f1 std:  0.0015
Cross-validation accuracy score:  0.5539
Cross-validation accuracy std:  0.0032
********************
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 9.06 µs


In [8]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# sgdc pipeline
sgdc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('sgdc', SGDClassifier())
])

# sgdc Parameter Grid 
# Using optimal parameters from previous gridsearch
param_grid = {
    'sgdc__loss': ['hinge'], #using only hinge loss in order to implement SVM
    'sgdc__penalty': ['l2'],
    'sgdc__learning_rate': ['adaptive'],
    'sgdc__alpha': [.5],
    'sgdc__eta0': [.025]
}

# Grid search
grid_search = GridSearchCV(estimator=sgdc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_41, y_train_41)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time


Best parameters found:  {'sgdc__alpha': 0.5, 'sgdc__eta0': 0.025, 'sgdc__learning_rate': 'adaptive', 'sgdc__loss': 'hinge', 'sgdc__penalty': 'l2'}
Cross-validation recall:  0.8654
Cross-validation recall std:  0.0053
Cross-validation precision:  0.1799
Cross-validation precision std:  0.0007
Cross-validation f1 score:  0.2978
Cross-validation f1 std:  0.0013
Cross-validation accuracy score:  0.5366
Cross-validation accuracy std:  0.0011
********************
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 11.7 µs


In [9]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# rfc pipeline
rfc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('rfc', RandomForestClassifier())
])

# sgdc Parameter Grid  
# Using optimal parameters from previous gridsearch
param_grid = {
    'rfc__n_estimators': [30],
    'rfc__criterion': ['gini'], #using gini to optimize run time
    'rfc__max_depth': [3],
    'rfc__random_state': [42]
}

# Grid search
grid_search = GridSearchCV(estimator=rfc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_41, y_train_41)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4)) 

print('********************')
%time

Best parameters found:  {'rfc__criterion': 'gini', 'rfc__max_depth': 3, 'rfc__n_estimators': 30, 'rfc__random_state': 42}
Cross-validation recall:  0.7859
Cross-validation recall std:  0.0051
Cross-validation precision:  0.1969
Cross-validation precision std:  0.0014
Cross-validation f1 score:  0.3149
Cross-validation f1 std:  0.0017
Cross-validation accuracy score:  0.6118
Cross-validation accuracy std:  0.0044
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 12.6 µs


## Top 30 Features

In [10]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# NB pipeline
nb_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('nb', GaussianNB())
])

# NB Parameter Grid
# Using optimal parameters from previous gridsearch
param_grid = {
    'nb__var_smoothing': [3.125], #using optimal parameter
}

# Grid search
grid_search = GridSearchCV(estimator=nb_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_30, y_train_30)

#print("Results Table: ", pd.DataFrame(grid_search.cv_results_))
#pd.set_option('display.max_columns',None)
print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time

Best parameters found:  {'nb__var_smoothing': 3.125}
Cross-validation recall:  0.8386
Cross-validation recall std:  0.0067
Cross-validation precision:  0.1821
Cross-validation precision std:  0.001
Cross-validation f1 score:  0.2992
Cross-validation f1 std:  0.0015
Cross-validation accuracy score:  0.5539
Cross-validation accuracy std:  0.0032
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [11]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# sgdc pipeline
sgdc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('sgdc', SGDClassifier())
])

# sgdc Parameter Grid 
# Using optimal parameters from previous gridsearch
param_grid = {
    'sgdc__loss': ['hinge'], #using only hinge loss in order to implement SVM
    'sgdc__penalty': ['l2'],
    'sgdc__learning_rate': ['adaptive'],
    'sgdc__alpha': [.5],
    'sgdc__eta0': [.025]
}

# Grid search
grid_search = GridSearchCV(estimator=sgdc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_30, y_train_30)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))


print('********************')
%time



Best parameters found:  {'sgdc__alpha': 0.5, 'sgdc__eta0': 0.025, 'sgdc__learning_rate': 'adaptive', 'sgdc__loss': 'hinge', 'sgdc__penalty': 'l2'}
Cross-validation recall:  0.8656
Cross-validation recall std:  0.0054
Cross-validation precision:  0.1799
Cross-validation precision std:  0.0007
Cross-validation f1 score:  0.2978
Cross-validation f1 std:  0.0013
Cross-validation accuracy score:  0.5366
Cross-validation accuracy std:  0.0011
********************
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 8.58 µs


In [12]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# rfc pipeline
rfc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('rfc', RandomForestClassifier())
])

# sgdc Parameter Grid  
# Using optimal parameters from previous gridsearch
param_grid = {
    'rfc__n_estimators': [30],
    'rfc__criterion': ['gini'], #using gini to optimize run time
    'rfc__max_depth': [3],
    'rfc__random_state': [42]
}

# Grid search
grid_search = GridSearchCV(estimator=rfc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_30, y_train_30)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4)) 

print('********************')
%time

Best parameters found:  {'rfc__criterion': 'gini', 'rfc__max_depth': 3, 'rfc__n_estimators': 30, 'rfc__random_state': 42}
Cross-validation recall:  0.7859
Cross-validation recall std:  0.0051
Cross-validation precision:  0.1969
Cross-validation precision std:  0.0014
Cross-validation f1 score:  0.3149
Cross-validation f1 std:  0.0017
Cross-validation accuracy score:  0.6118
Cross-validation accuracy std:  0.0044
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.78 µs


## Top 20 Features

In [13]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# NB pipeline
nb_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('nb', GaussianNB())
])

# NB Parameter Grid
# Using optimal parameters from previous gridsearch
param_grid = {
    'nb__var_smoothing': [3.125], #using optimal parameter
}

# Grid search
grid_search = GridSearchCV(estimator=nb_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_20, y_train_20)

#print("Results Table: ", pd.DataFrame(grid_search.cv_results_))
#pd.set_option('display.max_columns',None)
print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time

Best parameters found:  {'nb__var_smoothing': 3.125}
Cross-validation recall:  0.8386
Cross-validation recall std:  0.0067
Cross-validation precision:  0.1821
Cross-validation precision std:  0.001
Cross-validation f1 score:  0.2992
Cross-validation f1 std:  0.0015
Cross-validation accuracy score:  0.5539
Cross-validation accuracy std:  0.0032
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [14]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# sgdc pipeline
sgdc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('sgdc', SGDClassifier())
])

# sgdc Parameter Grid 
# Using optimal parameters from previous gridsearch
param_grid = {
    'sgdc__loss': ['hinge'], #using only hinge loss in order to implement SVM
    'sgdc__penalty': ['l2'],
    'sgdc__learning_rate': ['adaptive'],
    'sgdc__alpha': [.5],
    'sgdc__eta0': [.025]
}

# Grid search
grid_search = GridSearchCV(estimator=sgdc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_20, y_train_20)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time


Best parameters found:  {'sgdc__alpha': 0.5, 'sgdc__eta0': 0.025, 'sgdc__learning_rate': 'adaptive', 'sgdc__loss': 'hinge', 'sgdc__penalty': 'l2'}
Cross-validation recall:  0.8655
Cross-validation recall std:  0.0052
Cross-validation precision:  0.1799
Cross-validation precision std:  0.0007
Cross-validation f1 score:  0.2978
Cross-validation f1 std:  0.0012
Cross-validation accuracy score:  0.5366
Cross-validation accuracy std:  0.0011
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 10 µs


In [15]:
#Column Transformer
cols_to_std=['Age Group','Num of Bad Mental Health Days','Years Since Last Checkup','Hours of Sleeping','Age Started Smoking',
 'Cigarettes per Day','Days Drinking','Income','BMI']

ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# rfc pipeline
rfc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('rfc', RandomForestClassifier())
])

# sgdc Parameter Grid  
# Using optimal parameters from previous gridsearch
param_grid = {
    'rfc__n_estimators': [30],
    'rfc__criterion': ['gini'], #using gini to optimize run time
    'rfc__max_depth': [3],
    'rfc__random_state': [42]
}

# Grid search
grid_search = GridSearchCV(estimator=rfc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_20, y_train_20)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4)) 

print('********************')
%time

Best parameters found:  {'rfc__criterion': 'gini', 'rfc__max_depth': 3, 'rfc__n_estimators': 30, 'rfc__random_state': 42}
Cross-validation recall:  0.7859
Cross-validation recall std:  0.0051
Cross-validation precision:  0.1969
Cross-validation precision std:  0.0014
Cross-validation f1 score:  0.3149
Cross-validation f1 std:  0.0017
Cross-validation accuracy score:  0.6118
Cross-validation accuracy std:  0.0044
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


## Top 10 Features

In [16]:
#Column Transformer for Top 10 features
cols_to_std=['Age Group','Num of Bad Mental Health Days','Hours of Sleeping','Days Drinking','Income','BMI']

In [17]:
ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# NB pipeline
nb_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('nb', GaussianNB())
])

# NB Parameter Grid
# Using optimal parameters from previous gridsearch
param_grid = {
    'nb__var_smoothing': [3.125], #using optimal parameter
}

# Grid search
grid_search = GridSearchCV(estimator=nb_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_10, y_train_10)

#print("Results Table: ", pd.DataFrame(grid_search.cv_results_))
#pd.set_option('display.max_columns',None)
print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time

Best parameters found:  {'nb__var_smoothing': 3.125}
Cross-validation recall:  0.8221
Cross-validation recall std:  0.0053
Cross-validation precision:  0.1854
Cross-validation precision std:  0.0005
Cross-validation f1 score:  0.3026
Cross-validation f1 std:  0.0008
Cross-validation accuracy score:  0.5697
Cross-validation accuracy std:  0.0021
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [18]:
ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# sgdc pipeline
sgdc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('sgdc', SGDClassifier())
])

# sgdc Parameter Grid 
# Using optimal parameters from previous gridsearch
param_grid = {
    'sgdc__loss': ['hinge'], #using only hinge loss in order to implement SVM
    'sgdc__penalty': ['l2'],
    'sgdc__learning_rate': ['adaptive'],
    'sgdc__alpha': [.5],
    'sgdc__eta0': [.025]
}

# Grid search
grid_search = GridSearchCV(estimator=sgdc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_10, y_train_10)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4))

print('********************')
%time


Best parameters found:  {'sgdc__alpha': 0.5, 'sgdc__eta0': 0.025, 'sgdc__learning_rate': 'adaptive', 'sgdc__loss': 'hinge', 'sgdc__penalty': 'l2'}
Cross-validation recall:  0.868
Cross-validation recall std:  0.0061
Cross-validation precision:  0.1775
Cross-validation precision std:  0.0008
Cross-validation f1 score:  0.2948
Cross-validation f1 std:  0.0014
Cross-validation accuracy score:  0.5284
Cross-validation accuracy std:  0.0004
********************
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


In [19]:
ct = ColumnTransformer([('std',StandardScaler(),cols_to_std)])


# rfc pipeline
rfc_pipeline = Pipeline([
    ('ct', ct),
    ('smote', SMOTE(random_state=42)),  
    ('rfc', RandomForestClassifier())
])

# sgdc Parameter Grid  
# Using optimal parameters from previous gridsearch
param_grid = {
    'rfc__n_estimators': [30],
    'rfc__criterion': ['gini'], #using gini to optimize run time
    'rfc__max_depth': [3],
    'rfc__random_state': [42]
}

# Grid search
grid_search = GridSearchCV(estimator=rfc_pipeline, param_grid=param_grid, cv=5, scoring=['recall','precision','f1','accuracy'],refit='recall')
grid_search.fit(X_train_10, y_train_10)

print("Best parameters found: ", grid_search.best_params_)
print("Cross-validation recall: ", np.round(grid_search.cv_results_['mean_test_recall'][grid_search.best_index_],4))
print("Cross-validation recall std: ", np.round(grid_search.cv_results_['std_test_recall'][grid_search.best_index_],4))
print("Cross-validation precision: ", np.round(grid_search.cv_results_['mean_test_precision'][grid_search.best_index_],4))
print("Cross-validation precision std: ", np.round(grid_search.cv_results_['std_test_precision'][grid_search.best_index_],4))
print("Cross-validation f1 score: ", np.round(grid_search.cv_results_['mean_test_f1'][grid_search.best_index_],4))
print("Cross-validation f1 std: ", np.round(grid_search.cv_results_['std_test_f1'][grid_search.best_index_],4))
print("Cross-validation accuracy score: ", np.round(grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_],4))
print("Cross-validation accuracy std: ", np.round(grid_search.cv_results_['std_test_accuracy'][grid_search.best_index_],4)) 

print('********************')
%time

Best parameters found:  {'rfc__criterion': 'gini', 'rfc__max_depth': 3, 'rfc__n_estimators': 30, 'rfc__random_state': 42}
Cross-validation recall:  0.8027
Cross-validation recall std:  0.0149
Cross-validation precision:  0.192
Cross-validation precision std:  0.0037
Cross-validation f1 score:  0.3098
Cross-validation f1 std:  0.0039
Cross-validation accuracy score:  0.5936
Cross-validation accuracy std:  0.0148
********************
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
